# Learned Inertial Odometry

The paper by G. Cioffi et al. in RA-L 2023 claims that the IMU-based pose estimate $\vec{p}$ of a flying drone can be improved with the help of a learning-based model that estimates the relative pose change $\Delta \vec{p}$ between the current and previous time steps and known thrust (force) commands. Their code is available at

 * https://github.com/uzh-rpg/learned_inertial_model_odometry/

The idea is good and intuitive, but their solution raises a few questions.

## Concerns about the solution

In principle, the actions (thrust commands) of a drone define how it moves and thus can provide an estimate of the drone's pose $\vec{p}_t$ at any time instant $t$. Such mapping from a sequence of action commands to the current pose can be learned, for example, using a neural network regressor. However, this requires that i) the starting conditions are the same (e.g. the drone in the same pose on the ground) and ii) the action sequence contains all actions from the beginning.

**Relative pose paradox**

Why Cioffi et al. do not estimate the actual pose given the executed actions (thrust commands) but only a relative pose?

One reason could be, that they want to avoid difficulties related to arbitrary length sequences, which would need using recurrent neural networks (RNNs) that are often difficult to train. Instead, they use only $T$ latest actions (thrust commands). Naturally, one cannot know the absolute pose if only partial information about "movements" is known. However, it sounds intuitively correct that the relative change occurred between the latest $a_T$ and the previous action $a_{T-1}$ can be estimated. However, again a few questions arise:

 * Why multiple previous actions $a_T, a_{T-1}, a_{T-2}, \ldots, a_i$ improve the estimate at time $t=T$ if not all of them are available anyway - estimator should need only the latest action $a_T$ for relative change, right?
 * If a sequence of previous actions are used, then why not all of them as obviously this information is needed (or above is correct) and in this case some information is lost?

Actually, in their work the system runs at 100Hz and the history buffer is 0.5 seconds so the number of samples T=50. That does not sound that much so again another question raises

 * Why not to use a vanilla multilayer perceptron (MLP) instead of the more complex "temporal convolutional network" (TCN) from this [webpage](https://github.com/locuslab/TCN)?

It is assumed that TCN becomes useful when sequences are truly long such as 10,000 or 100,000 samples and in the TCN paper the performance is not that much better than standard RNNs - so why all this hassle?

**Even the relative pose estimation needs the full action history**

What makes the proposed solution even more questionable is that *under unknown physics it cannot be guaranteed that relative pose can be estimated from incomplete history of actions!*

## Proof of the claim
To illustrate the above claim that full action sequence is needed OR at least information about the sequence such as its length, we run some experiments on the simple Gymnasium environment *Pendulum*

<div><img src="images/pendulum.png"/></div>

for which the description is available at the Gymnasium [homepage](https://gymnasium.farama.org/environments/classic_control/pendulum/). However, these experiments should be easy to replicate with any other Gym environment.


Create the environment - in this case we assume that the current observation (the state of the environment) is the pose to be estimated.

In [ ]:
import gymnasium as gym
import numpy as np
env = gym.make('Pendulum-v1', g=9.81)

### Experiment 1: Train MLP using action-pose (observation) pairs

Our model is single input - single output model $\vec{p}_t = f(a_t)$ i.e. given the latest action, estimate the current absolute pose.

This is clearly impossible in many cases. For example, assume that the only action is either 0:"do nothing" or 1:"turn 5 degrees right". We cannot know the absolute pose (e.g. "pointing toward 25 degrees to the right") if we don't know that the previous four commands where (1,1,1,1).

However, let's verify this experimentally.

Collect training data of N samples. If the environment is terminated, we just restart it (it starts from an arbitrary initial conditions). Actions are 1-dimensional and state p is 3-dimensional.

**Note:** the pose p(T) corresponds to pose after action a(T) so the initial pose is missing (stored in p_0)

In [ ]:
N_tr = 1000

observation, info = env.reset()

p_0 = observation
a_l = np.empty((N_tr,1))
p_l = np.empty((N_tr,3))

num_of_e = 1
for s in range(N_tr):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    action = [0]
    observation, reward, terminated, truncated, info = env.step(action)
    a_l[s] = action
    p_l[s,:] = observation
    
    if terminated or truncated:
        observation, info = env.reset()
        num_of_e += 1
print(f'Contains data from {num_of_e} episodes')

Construct a simple MLP

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='sigmoid'),
  tf.keras.layers.Dense(3)
])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn)

Train the model for some epochs

In [ ]:
history = model.fit(a_l,p_l, epochs=5)

Plot the loss to make sure model is learning

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])

Run one episode and store N_te samples for testing

In [ ]:
N_te = 100

observation, info = env.reset()

p_0_test = observation 
a_l_test = np.empty((N_te,1))
p_l_test = np.empty((N_te,3))

for s in range(N_te):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    a_l_test[s] = action
    p_l_test[s,:] = observation
    
    if terminated or truncated:
        break

print(f'{s+1} samples generated for testing')
env.close()

Compute predictions

In [ ]:
p_hat = model.predict(a_l_test)

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1,3,1)
plt.plot(range(s), p_l_test[:s,0], range(s), p_hat[:s,0])
plt.title('x')
plt.legend(['gt','predicted'])
plt.subplot(1,3,2)
plt.title('y')
plt.plot(range(s), p_l_test[:s,1], range(s), p_hat[:s,1])
plt.subplot(1,3,3)
plt.title('Angular velocity')
plt.plot(range(s), p_l_test[:s,2], range(s), p_hat[:s,2])
plt.show()

The results show that nothing works. The estimated pose cannot be correct since

 * Not even the first estimate cannot be correct as the environment always restarts from random pose
 * All history of actions is lost

## Experiment 2: Train MLP using action - relative pose change

Learning $\Delta p_t = f(a_t)$ makes more sense since intuitively the current action affects a "change" to the current pose. The current pose can then be estimated by summing all relative pose changes to the initial pose $p = p_0+\sum_t \Delta p_t$.

Construct training data of relative poses

In [ ]:
N_tr = 1000

observation, info = env.reset()

p_0 = observation
a_l = np.empty((N_tr,1))
p_l = np.empty((N_tr,3))
p_delta_l = np.empty((N_tr,3))

prev_observation = observation
num_of_e = 1
for s in range(N_tr):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    action = [0]
    observation, reward, terminated, truncated, info = env.step(action)
    a_l[s] = action
    p_l[s,:] = observation
    p_delta_l[s,:] = observation-prev_observation
    
    if terminated or truncated:
        observation, info = env.reset()
        num_of_e += 1
    prev_observation = observation
print(f'Contains data from {num_of_e} episodes')

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='sigmoid'),
  tf.keras.layers.Dense(3)
])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn)

In [ ]:
history = model.fit(a_l,p_delta_l, epochs=5)

In [ ]:
plt.plot(history.history['loss'])

Run episode for testing

In [ ]:
N_te = 100

observation, info = env.reset()

p_0_test = observation 
a_l_test = np.empty((N_te,1))
p_l_test = np.empty((N_te,3))

for s in range(N_te):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    a_l_test[s] = action
    p_l_test[s,:] = observation
    
    if terminated or truncated:
        break

print(f'{s+1} samples generated for testing')
env.close()

Add the starting point and compute cumulative sum of deltas

In [ ]:
p_delta_hat = model.predict(a_l_test)
p_hat = p_0_test+np.cumsum(p_delta_hat, axis=0)

In [ ]:
plt.subplot(1,3,1)
plt.plot(range(s), p_l_test[:s,0], range(s), p_hat[:s,0])
plt.title('x')
plt.legend(['gt','predicted'])
plt.subplot(1,3,2)
plt.title('y')
plt.plot(range(s), p_l_test[:s,1], range(s), p_hat[:s,1])
plt.subplot(1,3,3)
plt.title('Angular velocity')
plt.plot(range(s), p_l_test[:s,2], range(s), p_hat[:s,2])
plt.show()

Results show that the first 1-2 estimates are close to the starting point, but otherwise estimates are wrong. For example, what happens next is very different for a pendulum swinging from left to right than right to left. And the first action has very little effect to that.

### Experiment 3: Fixing the starting point for delta pose estimation

This starts to make sense, since the same actions have always the same effect if we always start from the same pose.

**Note:** Instead of long sequences, let's use only short sequences as actual pose depends on action history and this still does not have idea what the previous actions have been.

In [ ]:
rand_seed = 42
N_tr = 1000
T_max = 5 # This many steps at most in every episode

observation, info = env.reset(seed=rand_seed)

p_0 = observation
a_l = np.empty((N_tr,1))
p_l = np.empty((N_tr,3))
p_delta_l = np.empty((N_tr,3))

prev_observation = observation
num_of_e = 1
for s in range(N_tr):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    action = [0]
    observation, reward, terminated, truncated, info = env.step(action)
    a_l[s] = action
    p_l[s,:] = observation
    p_delta_l[s,:] = observation-prev_observation
    
    if terminated or truncated or (s % T_max) == 0:
        observation, info = env.reset(seed=rand_seed)
        num_of_e += 1
    prev_observation = observation
print(f'Contains data from {num_of_e} episodes')

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='sigmoid'),
  tf.keras.layers.Dense(3)
])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn)

In [ ]:
history = model.fit(a_l,p_delta_l, epochs=5)

In [ ]:
plt.plot(history.history['loss'])

Generate training data

**Note:** You must generate several times as sometimes the result is good and sometimes not (note the randomness in action selection)

In [ ]:
N_te = 50

observation, info = env.reset(seed=rand_seed)

p_0_test = observation 
a_l_test = np.empty((N_te,1))
p_l_test = np.empty((N_te,3))

for s in range(N_te):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    a_l_test[s] = action
    p_l_test[s,:] = observation
    
    if terminated or truncated:
        break

print(f'{s+1} samples generated for testing')
env.close()

In [ ]:
p_delta_hat = model.predict(a_l_test)
p_hat = p_0_test+np.cumsum(p_delta_hat, axis=0)

In [ ]:
plt.subplot(1,3,1)
plt.plot(range(s), p_l_test[:s,0], range(s), p_hat[:s,0])
plt.title('x')
plt.legend(['gt','predicted'])
plt.subplot(1,3,2)
plt.title('y')
plt.plot(range(s), p_l_test[:s,1], range(s), p_hat[:s,1])
plt.subplot(1,3,3)
plt.title('Angular velocity')
plt.plot(range(s), p_l_test[:s,2], range(s), p_hat[:s,2])
plt.show()

Now the estimates start to look more meaningful. Since the pose always starts from the same initial pose, the first estimates are close to correct. However, the quality degrades the longer time from the beginning. This is because the estimate does not have an idea how many time steps have been taken (the longer time the more random actions). Therefore it is also difficult to make this predictor work well if the sequence lenght substantially increases.

**Note:** This would improve the more history inputs added, BUT that only moves drifting away further, not solve it.

### Experiment 4: Giving the predictor sense of time

The accumulation of time should aid the predictor to estimate what has happened so far and thus be much better in predicting how the current action will change the pose. Note that this cannot be accurate as actions have been random so that also affects to the current pose and also what happens next.

In [ ]:
rand_seed = 42
N_tr = 2000
T_max = 20 # This many steps at most in every episode

observation, info = env.reset(seed=rand_seed)

p_0 = observation
a_l = np.empty((N_tr,1))
p_l = np.empty((N_tr,3))
p_delta_l = np.empty((N_tr,3))
t_l = np.empty((N_tr,1))

prev_observation = observation
num_of_e = 1
steps = 0
for s in range(N_tr):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    action = [0]
    observation, reward, terminated, truncated, info = env.step(action)
    a_l[s] = action
    p_l[s,:] = observation
    p_delta_l[s,:] = observation-prev_observation
    t_l[s] = steps
    
    if terminated or truncated or (s % T_max) == 0:
        observation, info = env.reset(seed=rand_seed)
        num_of_e += 1
        steps = 0
    else:
        steps += 1
        
    prev_observation = observation
print(f'Contains data from {num_of_e} episodes')

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='sigmoid'),
  tf.keras.layers.Dense(3)
])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn)

In [ ]:
X = np.hstack([a_l,t_l])

history = model.fit(X,p_delta_l, epochs=10)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
N_te = 50

observation, info = env.reset(seed=rand_seed)

p_0_test = observation 
a_l_test = np.empty((N_te,1))
p_l_test = np.empty((N_te,3))
t_l_test = np.empty((N_te,1))

steps = 0
for s in range(N_te):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    a_l_test[s] = action
    p_l_test[s,:] = observation
    t_l_test[s] = steps

    steps += 1
    if terminated or truncated:
        break

print(f'{s+1} samples generated for testing')
env.close()

In [ ]:
X_te = np.hstack([a_l_test,t_l_test])

p_delta_hat = model.predict(X_te)
p_hat = p_0_test+np.cumsum(p_delta_hat, axis=0)

In [ ]:
plt.subplot(1,3,1)
plt.plot(range(s), p_l_test[:s,0], range(s), p_hat[:s,0])
plt.title('x')
plt.legend(['gt','predicted'])
plt.subplot(1,3,2)
plt.title('y')
plt.plot(range(s), p_l_test[:s,1], range(s), p_hat[:s,1])
plt.subplot(1,3,3)
plt.title('Angular velocity')
plt.plot(range(s), p_l_test[:s,2], range(s), p_hat[:s,2])
plt.show()